In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from nengo.utils.matplotlib import rasterplot

%matplotlib inline

In [2]:
af = scipy.io.loadmat("c1p8.mat")
stim = af["rho"]

In [3]:
def synthetic_neuron(drive):
    """
    Simulates a mock neuron with a time step of 1ms.
    Arguments:
    drive - input to the neuron (expect zero mean; SD=1)
    Returns:
    rho - response function (0=non-spike and 1=spike at each time step)
    """
    dt = .001
    T = dt*len(drive)
    time = np.arange(0, T, dt)
    lagSteps = .02/dt
    drive = np.concatenate((np.zeros(lagSteps), drive[lagSteps:]))
    system = scipy.signal.lti([1], [.03**2, 2*.03, 1])
    _, L, _ = scipy.signal.lsim(system, drive[:,np.newaxis], time)
    rate = np.divide(30, 1 + np.exp(50*(.05-L)))
    spikeProb = rate*dt
    return np.random.rand(len(spikeProb)) < spikeProb

In [4]:
af.keys()

['stim', '__version__', '__header__', 'rho', '__globals__']